# XGBoost

In [ ]:
from Functions import get_result, plot_precision_recall_vs_threshold, plot_roc_curve
from sklearn.metrics import confusion_matrix, make_scorer, recall_score, precision_score, f1_score
from sklearn.model_selection import RandomizedSearchCV
from xgboost import XGBClassifier
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

Load the datasets for training (remember they are scaled)

In [19]:
filenames = ['X_train.pkl', 'Y_train.pkl', 'X_val.pkl', 'Y_val.pkl', 'X_test.pkl', 'Y_test.pkl']
data_folder = '../data/'
dataframes = [pd.read_pickle(data_folder + filename) for filename in filenames]
X_train, Y_train, X_val, Y_val, X_test, Y_test = dataframes

In [20]:
# The scores we want to look at
scoring = {
    'accuracy': 'accuracy',
    'precision': make_scorer(precision_score, zero_division=0),
    'recall': make_scorer(recall_score, zero_division=0),
    'f1': make_scorer(f1_score, zero_division=0)
}

#### Hyperparameter Tuning

In [21]:
# We'll need the count of each class for a hyperparameter
risk_count = Y_train.value_counts()

In [22]:
param_grid = {
    'n_estimators': [100, 250, 500, 1000],      # Number of trees
    'max_depth': [5, 10, 25, 75],               # Maximum depth of a tree
    'learning_rate': [0.1, 0.5, 1, 2.5],        # Step size shrinkage used in update to prevent overfitting
    'objective': ['binary:logistic'],           # The learning task and the corresponding learning objective
    'eval_metric': ['aucpr'],                   # Evaluation metrics for validation data
    'scale_pos_weight': [1, 100, 250, 500,      # Control the balance of positive and negative weights
                         risk_count[0]/risk_count[1]] # Typical value to consider
}

In [23]:
# Initialize the RandomForestClassifier
xgb = XGBClassifier()

# Initialize RandomizedSearchCV
xgb_random = RandomizedSearchCV(estimator=xgb, param_distributions=param_grid, 
                               n_iter=50, cv=3, verbose=1, random_state=42, n_jobs=-1,
                               scoring=scoring, refit='f1')


In [24]:
xgb_random.fit(X_train, Y_train)

Fitting 3 folds for each of 50 candidates, totalling 150 fits


KeyboardInterrupt: 

In [ ]:
# Look at the results of the hyperparameter tuning
xgb_random_result_df = pd.DataFrame(xgb_random.cv_results_['params'])
xgb_random_result_df['Mean_Accuracy'] = xgb_random.cv_results_['mean_test_accuracy']
xgb_random_result_df['Mean_Recall'] = xgb_random.cv_results_['mean_test_recall']
xgb_random_result_df['Mean_Precision'] = xgb_random.cv_results_['mean_test_precision']
xgb_random_result_df['Mean_f1'] = xgb_random.cv_results_['mean_test_f1']
xgb_random_result_df['Rank_Recall'] = xgb_random.cv_results_['rank_test_recall']
xgb_random_result_df['Rank_Precision'] = xgb_random.cv_results_['rank_test_precision']
xgb_random_result_df['Rank_F1'] = xgb_random.cv_results_['rank_test_f1']
xgb_random_result_df.sort_values('Rank_F1')

,scale_pos_weight,objective,n_estimators,max_depth,learning_rate,eval_metric,Mean_Accuracy,Mean_Recall,Mean_Precision,Mean_f1,Rank_Recall,Rank_Precision,Rank_F1
47,708.316008,binary:logistic,500,10,1.0,aucpr,0.997216,0.393229,0.223354,0.284750,4,8,1
41,500.000000,binary:logistic,500,10,1.0,aucpr,0.997208,0.388021,0.221015,0.281494,6,9,2
22,500.000000,binary:logistic,1000,10,1.0,aucpr,0.997208,0.388021,0.221015,0.281494,6,9,2
29,500.000000,binary:logistic,100,10,1.0,aucpr,0.997205,0.388021,0.220790,0.281334,6,11,4
0,250.000000,binary:logistic,250,5,1.0,aucpr,0.997142,0.393229,0.217004,0.279533,4,17,5
14,250.000000,binary:logistic,500,5,1.0,aucpr,0.997179,0.388021,0.218494,0.279375,6,13,6
19,708.316008,binary:logistic,100,10,0.5,aucpr,0.997175,0.388021,0.218164,0.279138,6,14,7
27,500.000000,binary:logistic,500,10,0.1,aucpr,0.997194,0.385417,0.218852,0.279047,12,12,8
49,250.000000,binary:logistic,1000,5,1.0,aucpr,0.997175,0.385417,0.217267,0.277744,12,16,9
15,708.316008,binary:logistic,1000,10,0.5,aucpr,0.997186,0.382812,0.217395,0.277117,14,15,10


In [ ]:
# Evaluate the Best Model
print(f"Best Parameters: {xgb_random.best_params_}")
best_xgb = xgb_random.best_estimator_

Best Parameters: {'scale_pos_weight': 708.3160083160083, 'objective': 'binary:logistic', 'n_estimators': 500, 'max_depth': 10, 'learning_rate': 1, 'eval_metric': 'aucpr'}


In [ ]:
# Save the best model
best_xgb.save_model('Saved models/xgb_best_model.json')

In [ ]:
# Predict on the test set
test_predictions = best_xgb.predict(X_test)

In [ ]:
plot_roc_curve(Y_test, test_predictions)

In [ ]:
best_threshold = plot_precision_recall_vs_threshold('Test', Y_test, test_predictions)

In [ ]:
recall_1, precision_1, f1_1 = get_result(Y_test, test_predictions, best_threshold)

Lets again merge the training and validation set and predict on the test set to see if the model can perform better with more training data.



In [ ]:
# Initialize the RandomForestClassifier with the best hyperparameters from the tuning phase
xgb = XGBClassifier(
    n_estimators = xgb_random.best_params_['n_estimators'],
    max_depth = xgb_random.best_params_['max_depth'],
    learning_rate = xgb_random.best_params_['learning_rate'],
    objective = xgb_random.best_params_['objective'],
    eval_metric = xgb_random.best_params_['eval_metric'],
    scale_pos_weight = xgb_random.best_params_['scale_pos_weight']
)

In [ ]:
# Train the model
xgb.fit(pd.concat([X_train, X_val]), pd.concat([Y_train, Y_val]))

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='aucpr', feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=10, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=500, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [ ]:
test_predictions = xgb.predict(X_test)

In [ ]:
plot_roc_curve(Y_test, test_predictions)

In [ ]:
best_threshold = plot_precision_recall_vs_threshold('Test', Y_test, test_predictions)

In [ ]:
recall_2, precision_2, f1_2 = get_result(Y_test, test_predictions, best_threshold)

In [ ]:
if f1_2 > f1_1:
    xgb.save_model('xgb_best_model.json')
    print('By training our model on more data, we achived a higher performance')
else:
    print('The first model we found, through hyperparameter tuning, is still our best model.')
    print('It had the following scores:')
    print('Recall       =', round(recall_1, 2))
    print('Precision    =', round(precision_1, 2))
    print('F1           =', round(f1_1, 2))

The first model we found, through hyperparameter tuning, is still our best model.
It had the following scores:
Recall       = 0.53
Precision    = 0.23
F1           = 0.32
